# Example: Iterative Linear Algebraic Equation (LAEs) Solvers
This example will familiarize students with developing and using iterative solvers for systems of Linear Algebraic Equations (LAEs). We'll consider two iterative solvers: the [Jacobi](https://en.wikipedia.org/wiki/Jacobi_method) and [Gauss-Siedel](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) methods

* [Jacobi's method](https://en.wikipedia.org/wiki/Jacobi_method) updates the estimated solution for all variables at the same time. Let the estimate of the value of $x_{i}$ at iteration $k$ be $\hat{x}_{i,k}$. Then, the solution at the next iteration $\hat{x}_{i,k+1}$ is given by:
$$
\begin{equation*}
\hat{x}_{i,k+1}=\frac{1}{a_{ii}}\bigl(b_{i}-\sum_{j=1,i}^{n}a_{ij}\hat{x}_{j,k}\bigr)\qquad{i=1,2,\cdots,n}
\end{equation*}
$$
* The [Gauss-Seidel method](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) updates the best estimate of $\hat{x}_{i}$ while processing equations $i=1,\cdots,n$. Let the estimate for variable $i$ at iteration $k$ be $\hat{x}_{i,k}$. Then, the solution at the next iteration $\hat{x}_{i,k+1}$ is given by:
$$
\begin{equation*}
\hat{x}_{i,k+1}=\frac{1}{a_{ii}}\bigl(b_{i}-\sum_{j=1}^{i-1}a_{ij}\hat{x}_{j,k+1}-\sum_{j=i+1}^{n}a_{ij}\hat{x}_{j,k}\bigr)\qquad{i=1,2,\cdots,n}
\end{equation*}
$$

### Learning objectives
* __Task 1__: Random Diagonally Dominate $\mathbf{A}$ and right-hand-side vector $\mathbf{b}$. In this task, we'll generate a random system matrix $\mathbf{A}$ that is diagonally dominant and a random right-hand side vector $\mathbf{b}$
* __Task 2__: Solve the LAEs using the  Jacobi and the Gauss-Seidel methods. In this task, we'll solve our system of random linear algebraic equations using the [Jacobi](https://en.wikipedia.org/wiki/Jacobi_method) and [Gauss-Siedel](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) methods
* __Task 3__: In this task, we'll compare the runtime performance of the different iterative approaches against the Gaussian elimination method implemented by the LinearAlgebra.jl package included with Julia using the BenchmarkTools.jl package

## Setup
This example may use external third-party packages. In [the `Include.jl` file](Include.jl), we load our codes to access them in the notebook, set some required paths for this example, and load any required external packages.

In [3]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-7/L7a`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-7/L7a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-7/L7a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-7/L7a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-7/L7a/Manifest.toml`


### Task 1: Random Diagonally Dominate $\mathbf{A}$ and right-hand-side vector $\mathbf{b}$
In this task, we'll generate a random system matrix $\mathbf{A}$ that is diagonally dominant and a random right-hand side vector $\mathbf{b}$. Diagonal dominance is a sufficient (but not necessary) condition for the convergence of an iterative method. A diagonally dominate system matrix $\mathbf{A}$ has the feature:
$$
\begin{equation*}
\sum_{j=1,i}^{n}\lvert{a_{ij}}\rvert<\lvert{a_{ii}}\rvert\qquad\forall{i}
\end{equation*}
$$

* Diagonal dominance is a matrix property where the absolute value of the diagonal element of each row is greater than or equal to the sum of the absolute values of the other elements in that row.
A matrix that satisfies this property is said to be diagonally dominant.
* Diagonal dominance is a sufficient (but not necessary) condition for convergence. 
However, this condition says nothing above the rate of convergence.

Let's start by specifying how many rows we have in the _square_ system matrix $\mathbf{A}$ in the `number_of_rows::Int64` variable:

In [5]:
number_of_rows = 5000;

Then generate a $n\times{n}$ random system matrix $\mathbf{A}$ and a $n\times{1}$ random vector $\mathbf{b}$, [using the `randn(...)` method](https://docs.julialang.org/en/v1/stdlib/Random/#Base.randn). We add some extra to the diagonal elements of the test system matrix $\mathbf{A}$ to ensure diagonal dominance.

In [7]:
A = randn(number_of_rows, number_of_rows) .+ 10*(number_of_rows)*diagm(rand(number_of_rows));
b = randn(number_of_rows);

In [8]:
A

5000×5000 Matrix{Float64}:
 1800.78          -0.580079       1.1726    …    -0.82357         1.23923
    1.71814    38064.3           -1.2547          1.23272         1.12559
    2.06745        0.675289   38084.3             1.4233         -1.18796
   -0.0298352     -1.95492       -1.09746        -1.27442         0.329921
    1.69487        0.499851       0.215997       -0.0858058       0.66427
    1.20407        0.56518        0.375227  …     0.856029        0.335293
   -0.308721       0.723846       1.31072        -0.168072        0.513132
    0.298434       1.22813       -0.314731        1.77329         0.676428
    1.64475       -1.81837        1.2778          0.00101372      1.65647
    1.41252       -1.03642        2.25244         0.597802       -1.39221
   -0.194539      -0.9836        -1.04602   …     0.975321        0.0311234
   -0.132746       0.0478798      0.603676        0.201996       -1.50507
   -2.29082        1.97035        0.446483       -1.1156         -0.640303
    

### Check: Is the system matrix $\mathbf{A}$ strictly diagonally dominant?
Before we continue to the solvers, let's verify the randomly generated system matrix $\mathbf{A}$ is actually diagonally dominant. We check every row of the matrix $\mathbf{A}$ and store the result of each test in the `ddcondition::Dict{Int64, Bool}` variable.

In [10]:
ddcondition = Dict{Int64,Bool}()
for i ∈ 1:number_of_rows
    aii = abs(A[i,i]);
    σ = 0.0;
    for j ∈ 1:number_of_rows
        if (i ≠ j)
            σ += abs(A[i,j]);
        end
    end
    ddcondition[i] = (aii > σ) ? true : false;
end

If any of the entries of the `ddcondition::Dict{Int64, Bool}` are `false`, then we fail this test:

In [12]:
(findall(x-> x == 0, ddcondition) |> isempty) == true

false

## Task 2: Solve the LAEs using the  Jacobi and the Gauss-Seidel methods
In this task, we'll solve our system of random linear algebraic equations using the [Jacobi](https://en.wikipedia.org/wiki/Jacobi_method) and [Gauss-Siedel](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) methods. First, we set an overall error criteria (stopping condition), a maximum number of iterations that we are allowed, and an initial solution guess.

In [14]:
xₒ = rand(number_of_rows); # initial condition
maxiterations = 100;
ϵ = 1e-6;

### Jacobi
We call [the `solve(...)` method](src/Solvers.jl) with the appropriate data, including the solver type we wish to use, which in this case is [the Jacobi method](https://en.wikipedia.org/wiki/Jacobi_method). We indicate this choice by passing [a `MyJacobiMethod` instance](src/Types.jl) to the solve routine.

In [16]:
dJM = solve(A,b,xₒ, ϵ = ϵ, maxiterations = maxiterations, algorithm = MyJacobiMethod())

Dict{Int64, Vector{Float64}} with 14 entries:
  5  => [-9.79737e-5, 2.75247e-5, -3.60475e-5, -6.59125e-5, 6.52995e-5, -0.0003…
  12 => [-0.000132096, 2.73379e-5, -3.66729e-5, -6.46698e-5, 6.65817e-5, -0.000…
  8  => [-0.000131999, 2.73384e-5, -3.66699e-5, -6.46739e-5, 6.65776e-5, -0.000…
  1  => [-0.0260736, 0.000995201, 0.000998382, 0.00191133, -8.84239e-5, 0.00621…
  0  => [0.970427, 0.976511, 0.942971, 0.55362, 0.108358, 0.386573, 0.631567, 0…
  6  => [-0.000126347, 2.72316e-5, -3.65475e-5, -6.46306e-5, 6.67855e-5, -0.000…
  11 => [-0.000132096, 2.73379e-5, -3.66729e-5, -6.46699e-5, 6.65817e-5, -0.000…
  9  => [-0.000132075, 2.73377e-5, -3.66724e-5, -6.46701e-5, 6.65819e-5, -0.000…
  3  => [0.00115209, 1.49431e-5, 2.56223e-5, -7.42999e-5, 8.32552e-5, -0.000149…
  7  => [-0.000131876, 2.73351e-5, -3.66526e-5, -6.46812e-5, 6.65796e-5, -0.000…
  4  => [-0.000133775, 2.73997e-5, -3.25692e-5, -6.57937e-5, 6.64564e-5, -0.000…
  13 => [-0.000132096, 2.73379e-5, -3.66729e-5, -6.46698e-5, 6.

#### Check: Did we meet the error condition for Jacobi?
Let's check if the [the Jacobi method](https://en.wikipedia.org/wiki/Jacobi_method) met the desired error criteria. In this case, we'll check the _maxium error at the last iteration_. We compute the error for each equation and then find the worst case. If this worst-case error is smaller than our error tolerance, we pass the test:

In [18]:
error = A*dJM[maximum(keys(dJM))] - b
@assert maximum(error) < ϵ

### Gauss-Seidel method
Similar to above, we call [the `solve(...)` method](src/Solvers.jl) with the appropriate data, including the solver type we wish to use, which in this case is [the Gauss-Siedel method](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method). We indicate this choice by passing [a `MyGaussSeidelMethod` instance](src/Types.jl) to the solve routine.

In [20]:
dGSM = solve(A,b,xₒ, ϵ = ϵ, maxiterations = maxiterations, algorithm = MyGaussSeidelMethod())

Dict{Int64, Vector{Float64}} with 10 entries:
  0 => [0.970427, 0.976511, 0.942971, 0.55362, 0.108358, 0.386573, 0.631567, 0.…
  4 => [-0.000174487, 3.01922e-5, -3.44988e-5, -6.77501e-5, 6.19732e-5, -0.0003…
  5 => [-0.000131023, 2.72599e-5, -3.66386e-5, -6.45585e-5, 6.66811e-5, -0.0003…
  6 => [-0.000132154, 2.73443e-5, -3.66712e-5, -6.46789e-5, 6.65683e-5, -0.0003…
  2 => [0.00723384, -3.65074e-5, 0.000174751, -4.7829e-5, 2.97289e-5, 0.0011265…
  7 => [-0.000132091, 2.73375e-5, -3.66729e-5, -6.46694e-5, 6.65823e-5, -0.0003…
  9 => [-0.000132096, 2.73379e-5, -3.66729e-5, -6.46698e-5, 6.65817e-5, -0.0003…
  8 => [-0.000132096, 2.73379e-5, -3.66729e-5, -6.46699e-5, 6.65817e-5, -0.0003…
  3 => [-0.000219896, 3.11189e-5, -6.66539e-6, -7.90624e-5, 8.52451e-5, -0.0004…
  1 => [-0.0260736, 0.00104018, 0.00106977, 0.00173386, 3.88981e-5, 0.00662074,…

#### Check: Did we meet the error condition for Gauss-Seidel?
Let's check if the [the Gauss-Siedel method](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) met the desired error criteria. In this case, we'll check the _maxium error at the last iteration_. We compute the error for each equation and then find the worst case. If this worst-case error is smaller than our error tolerance, we pass the test:

In [22]:
error = A*dGSM[maximum(keys(dGSM))] - b
@assert maximum(error) < ϵ

## Task 3: How well do these algorithms scale?
In this task, we'll compare the runtime performance of the different iterative approaches against [the Gaussian elimination method](https://en.wikipedia.org/wiki/Gaussian_elimination) implemented by the [LinearAlgebra.jl package included with Julia](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#man-linalg) using [the BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl). We expect, in general [that the Gaussian elimination method](https://en.wikipedia.org/wiki/Gaussian_elimination) should be faster than the two iterative methods. 

#### Jacobi

In [25]:
let
    @benchmark solve(A,b,xₒ, ϵ = 1e-6, maxiterations = 100, algorithm = MyJacobiMethod()) setup=(A=$A,b=$b,xₒ=$xₒ)
end

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.357 s …   1.384 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.362 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.366 s ± 12.731 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                  ▁                                    ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.36 s         Histogram: frequency by time        1.38 s <

 Memory estimate: 2.03 MiB, allocs estimate: 113.

#### Gauss-Seidel

In [27]:
let
    @benchmark solve(A,b,xₒ, ϵ = 1e-6, maxiterations = 100, algorithm = MyGaussSeidelMethod()) setup=(A=$A,b=$b,xₒ=$xₒ)
end

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  992.094 ms …  1.015 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     999.574 ms             ┊ GC (median):    0.00%
 Time  (mean ± σ):      1.002 s ± 9.131 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █         █        █                    █                 █  
  █▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  992 ms          Histogram: frequency by time         1.01 s <

 Memory estimate: 1.07 MiB, allocs estimate: 60.

#### Gaussian elimination

In [29]:
let
    @benchmark solve(A,b,xₒ, ϵ = 1e-6, maxiterations = 100, algorithm = MyGaussianEliminationMethod()) setup=(A=$A,b=$b,xₒ=$xₒ)
end

BenchmarkTools.Trial: 16 samples with 1 evaluation.
 Range (min … max):  294.852 ms … 362.553 ms  ┊ GC (min … max): 0.00% … 13.13%
 Time  (median):     312.079 ms               ┊ GC (median):    0.49%
 Time  (mean ± σ):   314.573 ms ±  14.946 ms  ┊ GC (mean ± σ):  1.72% ±  3.21%

  ▁ ▁      ▁ ▁█ █ ▁  ▁█▁▁  ▁                                  ▁  
  █▁█▁▁▁▁▁▁█▁██▁█▁█▁▁████▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  295 ms           Histogram: frequency by time          363 ms <

 Memory estimate: 190.81 MiB, allocs estimate: 6.